In [41]:
import pandas as pd
import numpy as np
import folium
import matplotlib.pyplot as plt
import plotly.express as px
import branca.colormap as cm
from IPython.display import HTML
from folium.plugins import MousePosition
from folium import MacroElement
from jinja2 import Template
import warnings
from folium import Popup
warnings.filterwarnings('ignore')

In [37]:
data_collar = pd.read_csv("datasets/Collar.csv")
data_litologi = pd.read_csv("datasets/Litologi.csv")
data_assay = pd.read_csv("datasets/Assay.csv")

In [3]:
baris, kolom = data_assay.shape
print("Jumlah baris data assay:", baris)

Jumlah baris data assay: 3216


In [4]:
baris, kolom = data_litologi.shape
print("Jumlah baris data litologi:", baris)

Jumlah baris data litologi: 3217


In [ ]:
data_collar['Hole_id'] = data_collar['Hole_id'].str.extract('(\d+)', expand=False).astype(int)
data_assay['Hole_id'] = data_assay['Hole_id'].str.extract('(\d+)', expand=False).astype(int)
data_litologi['hole_id'] = data_litologi['hole_id'].str.extract('(\d+)', expand=False).astype(int)

,Hole_id,Samp_id,dept_from,dept_to,Ni,Co,Fe,SiO2,MgO
0,1,NaN,0.00,1.00,0.50,0.05,46.20,2.40,1.50
1,1,NaN,1.00,2.00,0.72,0.05,44.40,2.60,1.60
2,1,NaN,2.00,3.00,0.89,0.06,44.80,3.00,1.60
3,1,NaN,3.00,4.00,0.98,0.07,45.40,2.70,1.30
4,1,NaN,4.00,5.00,0.90,0.06,43.90,2.70,1.30
...,...,...,...,...,...,...,...,...,...
3211,125,NaN,10.00,10.85,1.11,0.14,42.60,17.00,3.70
3212,125,NaN,10.85,11.00,0.69,0.04,6.50,42.10,34.80
3213,125,NaN,11.00,12.00,0.95,0.06,8.22,40.26,33.58
3214,125,NaN,12.00,13.00,0.40,0.02,6.79,40.00,35.63


In [6]:
new_data = pd.concat([
    data_assay[['Hole_id']],
    data_litologi[['dept_from', 'Dept_to', 'Litho']],
    data_assay[[' Ni']]
], axis= 1)
new_data.to_csv('datasets/new_data.csv', index=False)
new_data

,Hole_id,dept_from,Dept_to,Litho,Ni
0,1.0,0.00,1.00,LIM,0.50
1,1.0,1.00,2.00,LIM,0.72
2,1.0,2.00,3.00,LIM,0.89
3,1.0,3.00,4.00,LIM,0.98
4,1.0,4.00,5.00,LIM,0.90
...,...,...,...,...,...
3212,125.0,10.00,10.85,SAP,0.69
3213,125.0,10.85,11.00,BR,0.95
3214,125.0,11.00,12.00,BR,0.40
3215,125.0,12.00,13.00,BR,0.39


In [7]:
sap_data = new_data[new_data['Litho'] == 'SAP'].copy()

sap_summary = sap_data.groupby('Hole_id').agg(
    Depth_from = ('dept_from', 'first'),
    Depth_to = ('Dept_to', 'last')
).reset_index()

ni_stats = sap_data.groupby('Hole_id').agg(
    Avg_Ni=(' Ni', 'mean'),
    Min_Ni=(' Ni', 'min'),
    Max_Ni=(' Ni', 'max')
).reset_index()

sap_summary['Total_SAP'] = sap_summary['Depth_to'] - sap_summary['Depth_from']

sap_summary = pd.merge(sap_summary, ni_stats, on='Hole_id', how='left')

sap_summary.to_csv('datasets/sap_summary.csv', index=False)

sap_summary

,Hole_id,Depth_from,Depth_to,Total_SAP,Avg_Ni,Min_Ni,Max_Ni
0,1.0,11.0,16.00,5.00,1.601667,1.03,2.08
1,2.0,9.0,24.00,15.00,2.294000,1.38,3.07
2,3.0,4.4,9.00,4.60,1.964000,0.83,2.53
3,4.0,8.0,13.00,5.00,1.614000,1.25,2.17
4,5.0,5.0,10.00,5.00,1.724000,1.50,2.13
...,...,...,...,...,...,...,...
120,121.0,10.0,17.00,7.00,1.635000,1.18,2.12
121,122.0,9.0,11.33,2.33,1.512000,0.66,2.10
122,123.0,8.0,14.00,6.00,2.520000,1.32,3.92
123,124.0,8.0,18.00,10.00,2.006154,0.60,2.93


In [8]:
plot_data = pd.concat([
    sap_summary[['Hole_id']],
    data_collar[['X', 'Y']],
    sap_summary[['Depth_from', 'Depth_to', 'Total_SAP', 'Avg_Ni', 'Min_Ni', 'Max_Ni']]
], axis= 1)
plot_data.to_csv('datasets/plot_data.csv', index=False)
plot_data

,Hole_id,X,Y,Depth_from,Depth_to,Total_SAP,Avg_Ni,Min_Ni,Max_Ni
0,1.0,-2.507508,121.513665,11.0,16.00,5.00,1.601667,1.03,2.08
1,2.0,-2.507511,121.512784,9.0,24.00,15.00,2.294000,1.38,3.07
2,3.0,-2.507511,121.511902,4.4,9.00,4.60,1.964000,0.83,2.53
3,4.0,-2.507515,121.510981,8.0,13.00,5.00,1.614000,1.25,2.17
4,5.0,-2.507488,121.510041,5.0,10.00,5.00,1.724000,1.50,2.13
...,...,...,...,...,...,...,...,...,...
120,121.0,-2.510636,121.507853,10.0,17.00,7.00,1.635000,1.18,2.12
121,122.0,-2.510652,121.508255,9.0,11.33,2.33,1.512000,0.66,2.10
122,123.0,-2.510583,121.508693,8.0,14.00,6.00,2.520000,1.32,3.92
123,124.0,-2.510650,121.509187,8.0,18.00,10.00,2.006154,0.60,2.93


In [10]:
# plt.figure(figsize=(15, 8))
# plt.scatter(data_collar['Y'], data_collar['X'], c='blue', alpha=0.5)
# plt.xlabel('Longitude')
# plt.ylabel('Latitude')
# plt.title('Peta titik dari data collar')
# plt.grid(True)
# plt.show()

In [76]:
lat = plot_data.iloc[0]['X']
lon = plot_data.iloc[0]['Y']
min_lat = plot_data['X'].min()
max_lat = plot_data['X'].max()
min_lon = plot_data['Y'].min()
max_lon = plot_data['Y'].max()

Min_Ni = plot_data['Min_Ni'].min()
Max_Ni = plot_data['Max_Ni'].max()

map = folium.Map(
    location=[lat, lon],
    zoom_start=17,
    control_scale= True,
    font_size='1.5rem'
    )

map.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])

colormap = cm.linear.YlOrRd_09.scale(Min_Ni, Max_Ni)
colormap.caption = 'Nickel Value Scale'
colormap.add_to(map)

mouse_position = MousePosition(
    position='bottomright',
    separator=' : ',
    prefix='Lat | Lon:',
    lat_formatter="function(num) {return L.Util.formatNum(num, 10);}",
    lng_formatter="function(num) {return L.Util.formatNum(num, 10);}",
)
mouse_position.add_to(map)


for _, row in plot_data.iterrows():
    SAP_val = row['Total_SAP']
    color = colormap(SAP_val)
    
    radius = 2 + SAP_val * 0.7

    folium.CircleMarker(
        location=[row['X'], row['Y']],  
        radius=radius,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=1,
        popup=Popup(
        html=(
            f"<b>Hole:</b> BH{row['Hole_id']:.0f}<br>"
            f"<b>Lat:</b> {row['X']:.5f}<br>"
            f"<b>Lon:</b> {row['Y']:.5f}<br>"
            f"<b>SAP:</b> {SAP_val:.2f} meter<br>"
            f"<b>Avg. Ni:</b> {row['Avg_Ni']:.2f}<br>"
            f"<b>Min. Ni:</b> {Min_Ni:.2f}<br>"
            f"<b>Max. Ni:</b> {Max_Ni:.2f}<br>"
        ),
        max_width=200  
    )
    ).add_to(map)
display(HTML('<div style="width: 1000px; height: 700px;">'+map._repr_html_()+'</div>'))

In [12]:
# map = folium.Map(zoom_starts = 15)

# locations = []
# for _, row in data_collar.iterrows():
#     location = [row['X'], row['Y']]
#     locations.append(location)
#     folium.CircleMarker(location=location, radius=3, color='blue', fill=True, fill_opacity=0.7).add_to(map)

# # Fit bounds ke semua lokasi
# map.fit_bounds(locations)
# map.save("collar_map.html")

In [13]:
# fig = px.scatter_mapbox(
#     data_collar,
#     lat="X",
#     lon="Y",
#     zoom=16,
#     height=600
# )

# fig.update_layout(mapbox_style="open-street-map")

# fig.show()